In [3]:
!pip install pydub

In [4]:
import os
import pandas as pd
from tqdm import tqdm
from pydub import AudioSegment
from IPython.display import Audio as AudioI

In [5]:
os.getcwd()

'/root/data/experiments/exp_2023-07-13_chunk_fixed'

In [6]:
os.chdir('../..')

In [7]:
os.getcwd()

'/root/data'

In [8]:
metadata = pd.read_csv('data/metadata-BACKUP.csv')
metadata_train = pd.read_csv('data/train/metadata.csv')
metadata_val = pd.read_csv('data/val/metadata.csv')

In [9]:
metadata.head()

,file_name,unique_file,path,species,label,subset,sample_rate,num_frames,length
0,Roeselianaroeselii_XC751814-dat028-019_edit1.wav,Roeselianaroeselii_XC751814-dat028-019,data/train/Roeselianaroeselii_XC751814-dat028-...,Roeselianaroeselii,56,train,44100,4586400,104.000000
1,Roeselianaroeselii_XC752367-dat006-010.wav,Roeselianaroeselii_XC752367-dat006-010,data/train/Roeselianaroeselii_XC752367-dat006-...,Roeselianaroeselii,56,train,44100,337571,7.654671
2,Yoyettacelis_GBIF2465208563_IN36000894_50988.wav,Yoyettacelis_GBIF2465208563_IN36000894_50988,data/train/Yoyettacelis_GBIF2465208563_IN36000...,Yoyettacelis,64,train,44100,220500,5.000000
3,Gomphocerippusrufus_XC752285-dat001-045.wav,Gomphocerippusrufus_XC752285-dat001-045,data/train/Gomphocerippusrufus_XC752285-dat001...,Gomphocerippusrufus,26,train,44100,693715,15.730499
4,Atrapsaltacorticina_GBIF2901504947_IN62966536_...,Atrapsaltacorticina_GBIF2901504947_IN62966536_...,data/val/Atrapsaltacorticina_GBIF2901504947_IN...,Atrapsaltacorticina,3,validation,44100,502740,11.400000


In [10]:
metadata_val.head()

,file_name,label
0,Atrapsaltacorticina_GBIF2901504947_IN62966536_...,3
1,Chorthippusbrunneus_XC751398-dat022-008_edit5.wav,10
2,Psaltodaplaga_GBIF3031797565_IN68469430_159997...,53
3,Omocestusviridulus_XC752267-dat013-003_edit2.wav,39
4,Omocestusviridulus_XC752263-dat012-007_edit1.wav,39


In [17]:
new_metadata = pd.DataFrame(columns=['file_name', 'unique_file', 'path', 'species',
                                    'label', 'subset', 'sample_rate', 'num_frames', 'length'])
new_metadata_train = pd.DataFrame(columns=['file_name', 'label'])
new_metadata_val = pd.DataFrame(columns=['file_name', 'label'])

In [20]:
threshold = 10

In [21]:
for i in tqdm(range(len(metadata))):
    file_name = metadata.iloc[[i]]['file_name'].values[0]
    unique_file = metadata.iloc[[i]]['unique_file'].values[0]
    path = metadata.iloc[[i]]['path'].values[0]
    species = metadata.iloc[[i]]['species'].values[0]
    label = metadata.iloc[[i]]['label'].values[0]
    subset = metadata.iloc[[i]]['subset'].values[0]
    sample_rate = metadata.iloc[[i]]['sample_rate'].values[0]
    num_frames = metadata.iloc[[i]]['num_frames'].values[0]
    length = metadata.iloc[[i]]['length'].values[0]
    audio = AudioSegment.from_wav(path)
    sec = 10000
    
    if subset == 'train':
        if length > threshold:
            n = int(length//10)
            for i in range(0, n+1):
                new_audio = audio[sec*i:sec*(i+1)]
                new_filename = f'{file_name[:-4]}'+'_cut'+str(i)+'.wav'
                new_path = f'data/data_chunks2/train/{new_filename}'
                new_len = len(new_audio) / 1000 
                new_frames = int(44100 * new_len)
                if new_len >= 1:
                    new_audio.export('data/data_chunks2/train/'+new_filename, format="wav")
                    new_metadata_train.loc[len(new_metadata_train)] = [new_filename, label]
                    new_path = f'data/data_chunks2/train/{new_filename}'
                    new_row = [new_filename, unique_file, new_path, species, label, subset,
                               sample_rate, new_frames, new_len]
                    new_metadata.loc[len(new_metadata)] = new_row
        else:
            audio.export('data/data_chunks2/train/'+file_name, format="wav")
            new_metadata_train.loc[len(new_metadata_train)] = [file_name, label]
            new_path = f'data/data_chunks2/train/{file_name}'
            new_row = [file_name, unique_file, new_path, species, label, subset,
                       sample_rate, num_frames, length]
            new_metadata.loc[len(new_metadata)] = new_row
    
    else:
        audio.export('data/data_chunks2/val/'+file_name, format="wav")
        new_metadata_val.loc[len(new_metadata_val)] = [file_name, label]
        new_path = f'data/data_chunks2/val/{file_name}'
        new_row = [file_name, unique_file, new_path, species, label, subset,
                   sample_rate, num_frames, length]
        new_metadata.loc[len(new_metadata)] = new_row

100%|██████████| 2331/2331 [05:16<00:00,  7.37it/s]


In [22]:
new_metadata

,file_name,unique_file,path,species,label,subset,sample_rate,num_frames,length
0,Roeselianaroeselii_XC751814-dat028-019_edit1_c...,Roeselianaroeselii_XC751814-dat028-019,data/data_chunks2/train/Roeselianaroeselii_XC7...,Roeselianaroeselii,56,train,44100,441000,10.00000
1,Roeselianaroeselii_XC751814-dat028-019_edit1_c...,Roeselianaroeselii_XC751814-dat028-019,data/data_chunks2/train/Roeselianaroeselii_XC7...,Roeselianaroeselii,56,train,44100,441000,10.00000
2,Roeselianaroeselii_XC751814-dat028-019_edit1_c...,Roeselianaroeselii_XC751814-dat028-019,data/data_chunks2/train/Roeselianaroeselii_XC7...,Roeselianaroeselii,56,train,44100,441000,10.00000
3,Roeselianaroeselii_XC751814-dat028-019_edit1_c...,Roeselianaroeselii_XC751814-dat028-019,data/data_chunks2/train/Roeselianaroeselii_XC7...,Roeselianaroeselii,56,train,44100,441000,10.00000
4,Roeselianaroeselii_XC751814-dat028-019_edit1_c...,Roeselianaroeselii_XC751814-dat028-019,data/data_chunks2/train/Roeselianaroeselii_XC7...,Roeselianaroeselii,56,train,44100,441000,10.00000
...,...,...,...,...,...,...,...,...,...
7016,Pholidopteralittoralis_XC752439-dat049-001_edi...,Pholidopteralittoralis_XC752439-dat049-001,data/data_chunks2/train/Pholidopteralittoralis...,Pholidopteralittoralis,44,train,44100,88200,2.00000
7017,Gomphocerussibiricus_XC751796-dat187-005_edit2...,Gomphocerussibiricus_XC751796-dat187-005,data/data_chunks2/train/Gomphocerussibiricus_X...,Gomphocerussibiricus,27,train,44100,441000,10.00000
7018,Gomphocerussibiricus_XC751796-dat187-005_edit2...,Gomphocerussibiricus_XC751796-dat187-005,data/data_chunks2/train/Gomphocerussibiricus_X...,Gomphocerussibiricus,27,train,44100,441000,10.00000
7019,Gomphocerussibiricus_XC751796-dat187-005_edit2...,Gomphocerussibiricus_XC751796-dat187-005,data/data_chunks2/train/Gomphocerussibiricus_X...,Gomphocerussibiricus,27,train,44100,308700,7.00000


In [29]:
new_metadata.to_csv('new_metadata.csv', index=False)
new_metadata_val.to_csv('new_metadata_val.csv', index=False)
new_metadata_train.to_csv('new_metadata_train.csv', index=False)